In [5]:
import base64
import socketio
import eventlet

import cv2 as cv
import numpy as np
import flask as F
from io import BytesIO
from PIL import Image

from keras.models import load_model

soi = socketio.Server()

app = F(__name__)
speed_limit = 10

@soi.on('telemetry')

def telemetry(sid, data):

    speed = float(data['speed'])
    image = Image.open(BytesIO(base64, b64decode(data['Image'])))
    image = np.asarray(image)

    image = cv.cvtColor(image, cv.COLOR_BG2YUV)
    image = image[65:135,:,:]

    image = cv.resize(image, (200,66))
    image = np.array([image])

    steering_angle = float(model.predict(image))
    throtte = 1.0 - speed/speed_limit
    print(steering_angle, throtte, speed)
    send_control(steering_angle, throtte)


def send_control(steering_angle, throtte):
    soi.emit('steer', data = {'steering_angle':steering_angle.__str__(), 'throtte':throtte.__str__()})

def connect(sid,environ):
    print('Connected')
    send_control(0,0)

if __name__ == '__main__':

    model = load_model('12_Epoch_Model.h5')
    ap = socketio.Middleware(soi,app)

    eventlet.wgsi.server(eventlet.listen('',4567)).app


TypeError: 'module' object is not callable